In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
%config Completer.use_jedi = False

# Importing Database from csv of Jakarta Postal Codes

In [3]:
df = pd.read_csv('JakartaPostalCodes.csv')

In [4]:
df.head()

,Kode Pos,Kecamatan,Kelurahan
0,12430,Cilandak,Cilandak Barat
1,12410,Cilandak,Cipete Selatan
2,12420,Cilandak,Gandaria Selatan
3,12440,Cilandak,Lebak Bulus
4,12450,Cilandak,Pondok Labu


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Kode Pos   65 non-null     int64 
 1   Kecamatan  65 non-null     object
 2   Kelurahan  65 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


In [6]:
len(df['Kelurahan'])

65

# Now lets Search each Latitude and Longitude of each neighborhood

In [7]:
geolocator = Nominatim(user_agent="jakartans_users")
location = geolocator.geocode("Jakarta Selatan")
print(location.latitude, location.longitude)
latjaksel = location.latitude
longjaksel = location.longitude


-6.28381815 106.80486324917382


In [8]:
longlat = []
for x in range(len(df['Kelurahan'])):
    location = geolocator.geocode("{},{}".format(df['Kelurahan'][x],df['Kecamatan'][x]))
    longlat.append([location.latitude, location.longitude])


In [9]:
longlat = np.array(longlat)


# As we finished getting our latitude and longitude for each neighborhood, its time to add it into dataframe

In [10]:
longlat = pd.DataFrame(longlat,columns=['Latitude','Longitude'])

In [11]:
longlat.head()

,Latitude,Longitude
0,-6.290266,106.796215
1,-6.272540,106.805768
2,-6.271562,106.795172
3,-6.302568,106.780354
4,-6.309144,106.797938


In [12]:
df = pd.concat([df,longlat],axis=1)

In [13]:
df.head()

,Kode Pos,Kecamatan,Kelurahan,Latitude,Longitude
0,12430,Cilandak,Cilandak Barat,-6.290266,106.796215
1,12410,Cilandak,Cipete Selatan,-6.272540,106.805768
2,12420,Cilandak,Gandaria Selatan,-6.271562,106.795172
3,12440,Cilandak,Lebak Bulus,-6.302568,106.780354
4,12450,Cilandak,Pondok Labu,-6.309144,106.797938


# Adding Population Data of each Neighborhood / Kelurahan into the dataset

In [28]:
dfpop = pd.read_csv('data-kependudukan-tahun-2019.csv')
dfpop.head()

,kecamatan,kelurahan,jumlah_lakilaki,jumlah_perempuan
0,TEBET,TEBET TIMUR,10473,11079
1,TEBET,TEBET BARAT,12505,12987
2,TEBET,MENTENG DALAM,22440,22481
3,TEBET,KEBON BARU,21562,21493
4,TEBET,BUKIT DURI,21103,20812


In [29]:
dfpop['kecamatan'] = dfpop['kecamatan'].str.title()
dfpop['kelurahan'] = dfpop['kelurahan'].str.title()
dfpop.head()

,kecamatan,kelurahan,jumlah_lakilaki,jumlah_perempuan
0,Tebet,Tebet Timur,10473,11079
1,Tebet,Tebet Barat,12505,12987
2,Tebet,Menteng Dalam,22440,22481
3,Tebet,Kebon Baru,21562,21493
4,Tebet,Bukit Duri,21103,20812
